In [1]:
import torch

In [13]:
seq_k = torch.tensor([[1, 2, 0, 0, 0, 0], 
                      [4, 5, 6, 0, 0, 0], 
                      [7, 8, 9, 10, 0, 0], 
                      [10, 11, 12, 13, 14, 0], 
                      [13, 14, 15, 0, 0, 0]])

seq_q = torch.tensor([[1, 2, 3, 4, 5, 0],
                        [4, 5, 6, 7, 8, 0],
                        [7, 8, 9, 10, 11, 0],
                        [10, 11, 12, 13, 14, 0],
                        [13, 14, 15, 16, 17, 0]])

In [14]:
batch_size, len_q = seq_q.size() # batch_size x len_q
batch_size, len_k = seq_k.size() # batch_size x len_k

In [15]:
print(seq_q.shape)
print(seq_k.shape)

torch.Size([5, 6])
torch.Size([5, 6])


In [16]:
seq_k.data.eq(0)

tensor([[False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False,  True,  True,  True]])

In [17]:
seq_k.data.eq(0).shape

torch.Size([5, 6])

In [22]:
pad_mask = seq_k.data.eq(0).unsqueeze(1)

In [23]:
pad_mask.shape

torch.Size([5, 1, 6])

In [25]:
padding_mask = pad_mask.expand(batch_size, len_q, len_k)

In [28]:
padding_mask

tensor([[[False, False,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True]],

        [[False, False, False,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False,  True,  True,  True]],

        [[False, False, False, False,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False,  True,  True]],

        [[False, False, False, False, Fa

In [37]:
padding_mask[:, 1, :]

tensor([[False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False,  True,  True,  True]])

In [38]:
padding_mask[:, 0, :]

tensor([[False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False,  True,  True,  True]])

In [39]:
look_ahead_mask = torch.ones(len_q, len_q).triu(1) # seq_len x seq_len


In [40]:
look_ahead_mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [41]:
look_ahead_mask.shape

torch.Size([6, 6])

In [45]:
look_ahead_masking = look_ahead_mask.unsqueeze(0).expand(batch_size, len_q, len_q)

In [46]:
look_ahead_masking

tensor([[[0., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1.],
         [0.